<a href="https://colab.research.google.com/github/LukeSky117/My_Project/blob/main/Ajuda_ao_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -U -q google-generativeai

In [17]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [16]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [18]:
title = 'Informação sobre a enchente no Rio Grande do Note e a união do povo na calamidade'
sample_text = ('Título: Informação sobre a enchente no Rio Grande do Note e a união do povo na calamidade'
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Situação Alarmante no RS: Uma chamada à ação.")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.0062000193, -0.022192113, -0.042079, 0.039808467, 0.024262223, 0.01695919, -0.050724424, -0.019097026, 0.028313901, 0.06911282, -0.02907881, 0.021124346, -0.03150926, -0.056057237, 0.010379922, -0.0069565754, -0.010414074, -0.02660059, -0.04593054, -0.053290468, 0.01211388, 0.021807501, -0.018539434, -0.01662878, -0.02091588, 0.005255028, -0.0019750397, -0.060051758, -0.0456912, 0.034681424, -0.045664504, 0.016234664, -0.049116082, 0.026481157, -0.011762106, -0.0889258, 0.0043155407, -0.016602766, 0.012587222, -0.016547376, 0.016320629, -0.068039104, -0.0019422369, 0.0040742536, -0.02608676, -0.015108629, 0.025638962, 0.04265355, 0.020736413, -0.060697377, 0.0110137025, -0.015043574, 0.108618, -0.0050713704, -0.012896987, -0.026669696, 0.0360397, 0.0076833726, -0.028613238, 0.010923547, 0.017547948, 0.04133651, 0.0023982783, 0.051268525, -0.019508803, -0.049822483, -0.03653834, -0.012606157, 0.03146416, -0.009847811, 0.02161454, -0.049182843, 0.03257962, -0.024732517, 

In [19]:
DOCUMENT1 = {
    "Título": "Imprudência e Falta de Ação",
    "Conteúdo": "A crise política no RS é marcada pela imprudência do governo, que tem falhado em proteger seus cidadãos. As ações, ou a falta delas, têm gerado consequências devastadoras, levando a população a buscar ajuda de outras regiões do país, de outros países e até mesmo de empresários. A falta de transparência e a incapacidade de lidar com a crise aumentam a desconfiança da população em relação aos líderes políticos."}

DOCUMENT2 = {
    "Título": "Indignação e Manipulação da Narrativa",
    "Conteúdo": "A crise social se intensifica a cada dia, com a população clamando por ajuda e expressando indignação com a falta de ação do governo. A mídia, em alguns casos, tenta amenizar a situação, apresentando apenas relatos selecionados e ignorando a realidade de muitos gaúchos. Vídeos e depoimentos de pessoas indignadas, clamando por nota fiscal para doações, denunciando o impedimento de ajuda humanitária e o aumento dos preços de produtos básicos produzidos no RS são abafados. Essa manipulação da narrativa agrava a crise e aumenta o sentimento de abandono da população."}

DOCUMENT3 = {
    "Título": "Solidariedade e Denúncia",
    "Conteúdo": "Em momentos de crise, a solidariedade é fundamental. Diversas iniciativas buscam levar ajuda aos gaúchos. Vaquinhas online, apoio de pessoas próximas e a ampla divulgação da situação, seja por influenciadores digitais ou por cidadãos comuns, são cruciais para mobilizar a sociedade. É preciso ir além da ajuda material. É fundamental denunciar a negligência do governo, a manipulação da mídia e exigir ações efetivas para combater a crise. A sociedade precisa se unir para pressionar por mudanças e garantir que os direitos da população gaúcha sejam respeitados."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [20]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Imprudência e Falta de Ação,A crise política no RS é marcada pela imprudên...
1,Indignação e Manipulação da Narrativa,"A crise social se intensifica a cada dia, com ..."
2,Solidariedade e Denúncia,"Em momentos de crise, a solidariedade é fundam..."


In [21]:
model = "models/embedding-001"

In [22]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [23]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Imprudência e Falta de Ação,A crise política no RS é marcada pela imprudên...,"[0.036740493, -0.06148031, -0.03803934, 0.0250..."
1,Indignação e Manipulação da Narrativa,"A crise social se intensifica a cada dia, com ...","[0.045476615, -0.025160942, -0.02492978, 0.038..."
2,Solidariedade e Denúncia,"Em momentos de crise, a solidariedade é fundam...","[0.014412624, -0.02259262, -0.029880894, 0.009..."


In [24]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [26]:
consulta = "Como faço para ajudar?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Em momentos de crise, a solidariedade é fundamental. Diversas iniciativas buscam levar ajuda aos gaúchos. Vaquinhas online, apoio de pessoas próximas e a ampla divulgação da situação, seja por influenciadores digitais ou por cidadãos comuns, são cruciais para mobilizar a sociedade. É preciso ir além da ajuda material. É fundamental denunciar a negligência do governo, a manipulação da mídia e exigir ações efetivas para combater a crise. A sociedade precisa se unir para pressionar por mudanças e garantir que os direitos da população gaúcha sejam respeitados.


In [29]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [28]:
prompt = f"Escreva um texto que ajude as pessoas na busca de  melhor informação: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Buscando a Verdade em Tempos de Crise: Um Guia para Encontrar Informações Confiáveis**

Em meio à crise atual, é essencial buscar informações precisas e confiáveis para tomar decisões informadas. Aqui estão algumas dicas para ajudá-lo a navegar no cenário da informação:

* **Verifique as fontes:** Confie em fontes confiáveis, como organizações de notícias respeitadas, agências governamentais e instituições acadêmicas. Evite fontes anônimas ou tendenciosas.
* **Leia além das manchetes:** Não se contente apenas com as manchetes. Leia os artigos completos para obter uma compreensão mais profunda do contexto e dos detalhes.
* **Procure perspectivas diversas:** Exponha-se a uma variedade de perspectivas para evitar viés de confirmação. Considere fontes que podem ter opiniões diferentes das suas.
* **Verifique os fatos:** Use ferramentas de verificação de fatos para confirmar a precisão das informações. Sites como Snopes e PolitiFact podem ajudá-lo a identificar informações falsas ou engan